In [4]:
import pandas as pd
import altair as alt
import wbgapi as wb

In [3]:
#pip install wbgapi

Note: you may need to restart the kernel to use updated packages.


In [5]:
happy_all = pd.read_csv("https://raw.githubusercontent.com/ccsuehara/python_verano/main/Clase%206/data/world-happiness-report.csv")
happy_21 = pd.read_csv("https://raw.githubusercontent.com/ccsuehara/python_verano/main/Clase%206/data/world-happiness-report-2021.csv")

happy = pd.merge(happy_all[happy_all['year'] == 2019],
                  happy_21[['Country name', "Regional indicator" ]],
                  on = "Country name",
                  how  = "inner",
                 indicator = True)

In [14]:

#alt.themes.enable('urbaninstitute')
#latimes
#ggplot2

# Definir un tema, más sobre esto: 
#https://towardsdatascience.com/consistently-beautiful-visualizations-with-altair-themes-c7f9f889602
def tema_graf():
    font = "Helvetica"
    labelFont = "Garamond"
    sourceFont = "Calibri"
    
    return {
        "config" : {
             "title": {'font': font, 
                       'fontSize': 15,
                       'fontWeight': 300,
                       'offset':0,
                       'anchor':'left'},
             "axis": {
                  "labelFont": font,
                  "titleFont": font,
                  "domainColor": "gray"
             },
             "header": {
                  "labelFont": font,
                  "titleFont": font
             },
             "legend": {
                  "labelFont": labelFont,
                  "titleFont": labelFont,
                  "symbolType": 'circle',
                  "padding": 1
                  
             }
             ,
             'background': '#d7dff4'             
        }
    }

alt.themes.register('mitema', tema_graf)
alt.themes.enable('mitema')

ThemeRegistry.enable('mitema')

In [13]:
# Poniendo título a nuestro gráfico 
alt.Chart(happy, height = 500, width = 800).mark_point().encode(
    x = alt.X("Life Ladder:Q"),
    y = alt.Y("Log GDP per capita:Q", scale = alt.Scale()),
    color = alt.Color("Regional indicator"),
    size = alt.Size('Generosity')
    ).interactive().properties(
    title={
      "text": ["Esto parece un gráfico inspirado en Rosling"], 
      "subtitle": ["Es un homenaje",
                   "Fuente: Happiness Report"],
      "color": "Black",
      "subtitleColor": "Black"
    })

alt.Chart(...)

In [ ]:
#Cambiando los titulos de los ejes

alt.Chart(happy).mark_point().encode(
    x = alt.X("Life Ladder:Q",  title = "índice de felicidad"),
    y = alt.Y("Log GDP per capita:Q" ,  title = "PBi per capita (logs)"),
    color = alt.Color("Regional indicator"),
    size = alt.Size('Generosity')
    ).interactive().properties(
    title={
      "text": ["Esto parece un gráfico inspirado en Rosling"], 
      "subtitle": ["Es un homenaje",
                   "Fuente: Happiness Report"],
      "color": "Black",
      "subtitleColor": "Red"
    })

In [ ]:
#Cambiando los titulos de los ejes
## Para ver los palettes disponibles https://vega.github.io/vega/docs/schemes/
alt.Chart(happy).mark_point().encode(
    x = alt.X("Life Ladder:Q",  title = "índice de felicidad"),
    y = alt.Y("Log GDP per capita:Q" ,  title = "PBi per capita (logs)"),
    color = alt.Color("Regional indicator"),
    size = alt.Size('Generosity')
    ).interactive().properties(
    title={
      "text": ["Esto parece un gráfico inspirado en Rosling"], 
      "subtitle": ["Es un homenaje",
                   "Fuente: Happiness Report"],
      "color": "Black",
      "subtitleColor": "Red"
    })

In [ ]:
#happy_peru = happy_all[happy_all['Country name'] == 'Peru']

In [ ]:
## Usando la api para descargar información del Banco mundial
## pip install wbgapi
## si no puedes usar pip, prueba hacer conda install pip
## Fuente: https://stackoverflow.com/questions/41060382/using-pip-to-install-packages-to-anaconda-environment

# tbl = wb.data.DataFrame('SP.POP.TOTL',
#                   time = range(2000, 2019), labels = True)

# tbl = tbl.reset_index()

# tbl.rename(columns = {'SP.POP.TOTL': 'poblacion'}, inplace = True)


lst = wb.economy.info().table()
lst.pop()
nombres = pd.DataFrame(data = lst, columns = ["economy", "Country", "Region", "income_classification"])
nombres.head(5)

tbl_f = pd.merge(tbl, nombres, on = ["economy", "Country"], how = "inner")

In [ ]:
tbl_f

In [ ]:
tbl_f = tbl_f.loc[tbl_f['Region'] != '']

In [ ]:
tbl_melt = tbl_f.melt( id_vars = ['economy', 'Country', 'Region', 'income_classification'],
                   value_vars = [col for col in tbl if col.startswith('YR')])

In [ ]:
tbl_melt['year'] = tbl_melt['variable'].str.replace('YR', '').astype('int')

tbl_melt.head(10)

In [ ]:

## agregaciones, y otros gráficos

g1 = alt.Chart(tbl_melt).mark_area().encode(
    x="year",
    y="sum(value):Q",
    color="Region:N"
)

g1

In [ ]:
peru = tbl_melt.loc[tbl_melt['Country'] == 'Peru']
peru.head()

In [ ]:
g2 = alt.Chart(tbl_melt).mark_bar().encode(
    x = alt.X("year"),
    y="value:Q",
    color=alt.condition(
        alt.datum.year == 2005,
        alt.value("red"),  # The positive color
        alt.value("blue")  # The negative color
    )
)

In [ ]:
g3 = alt.Chart(tbl_melt).mark_area().encode(
    x='year:Q',
    y='sum(value):Q',
    color='Region:N',
    row = alt.Row('Region:N')
).properties(height=50, width=800)

g3

In [ ]:
alt.hconcat(g1, g2)

In [ ]:
alt.vconcat(alt.hconcat(g1, g2), g3)